In [1]:
import pandas as pd
from clickhouse_driver import Client
from datetime import datetime,date,timedelta
import time
import json

# create datetime dimension

In [615]:
def create_date_dimension(start, end):
    df = pd.DataFrame({"date": pd.date_range(start, end,freq='D')})
    df["week_day"] = df.date.dt.day_name()
    df["day"] = df.date.dt.day
    df["month"] = df.date.dt.month
    df["year"] = df.date.dt.year
    df["week"] = df.date.dt.weekofyear
    df["quarter"] = df.date.dt.quarter
    df.insert(0, 'date_id', (df.year.astype(str) + df.month.astype(str).str.zfill(2) + df.day.astype(str).str.zfill(2)).astype(int))
    return df

In [28]:
start = date(2020,1,1)
end = date(2023,2 ,1)

In [31]:
create_date_table(start, end).columns

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_24052/1492513383.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["week"] = df.date.dt.weekofyear


Index(['date_id', 'date', 'week_day', 'day', 'month', 'year', 'week',
       'quarter'],
      dtype='object')

In [32]:
create_date_table(start, end).head(5)

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_24052/1492513383.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["week"] = df.date.dt.weekofyear


,date_id,date,week_day,day,month,year,week,quarter
0,20200101,2020-01-01,Wednesday,1,1,2020,1,1
1,20200102,2020-01-02,Thursday,2,1,2020,1,1
2,20200103,2020-01-03,Friday,3,1,2020,1,1
3,20200104,2020-01-04,Saturday,4,1,2020,1,1
4,20200105,2020-01-05,Sunday,5,1,2020,1,1


create table in powerBi

In [33]:
{'name': 'date', 'columns':
 [{ 'name': 'date_id', 'dataType': 'Int64'},
  { 'name': 'date', 'dataType': 'DateTime'},
  { 'name': 'week_day', 'dataType': 'string'},
  { 'name': 'day', 'dataType': 'Int64'},
  { 'name': 'month', 'dataType': 'Int64'},
  { 'name': 'year', 'dataType': 'Int64'},
  { 'name': 'week', 'dataType': 'Int64'},
  { 'name': 'quarter', 'dataType': 'Int64'}
]
  }
    
      

{'name': 'date',
 'columns': [{'name': 'date_id', 'dataType': 'Int64'},
  {'name': 'date', 'dataType': 'DateTime'},
  {'name': 'week_day', 'dataType': 'string'},
  {'name': 'day', 'dataType': 'Int64'},
  {'name': 'month', 'dataType': 'Int64'},
  {'name': 'year', 'dataType': 'Int64'},
  {'name': 'week', 'dataType': 'Int64'},
  {'name': 'quarter', 'dataType': 'Int64'}]}

# time 

In [37]:
df = pd.DataFrame({"time":pd.date_range("00:00:00", "23:59:59", freq="1S").time})
    

In [2]:
def get_hour(time):
    hour = time.strftime('%H')
    return hour

In [3]:
def get_second(time):
    second = time.strftime('%S')
    return second

In [4]:
def get_minute(time):
    minute = time.strftime('%M')
    return minute

In [48]:
df['hour'] = df['time'].apply(get_hour)

In [54]:
df['minute'] = df['time'].apply(get_minute)

In [55]:
df['second'] = df['time'].apply(get_second)

In [5]:
def create_time_dimension():
    df = pd.DataFrame({"time":pd.date_range("00:00:00", "23:59:59", freq="1S").time})
    df['hour'] = df['time'].apply(get_hour)
    df['minute'] = df['time'].apply(get_minute)
    df['second'] = df['time'].apply(get_second)
    return df

In [58]:
create_time_dimension()

,time,hour,minute,second
0,00:00:00,00,00,00
1,00:00:01,00,00,01
2,00:00:02,00,00,02
3,00:00:03,00,00,03
4,00:00:04,00,00,04
...,...,...,...,...
86395,23:59:55,23,59,55
86396,23:59:56,23,59,56
86397,23:59:57,23,59,57
86398,23:59:58,23,59,58


In [59]:
{'name': 'date', 'columns':
 [{ 'name': 'time', 'dataType': 'time'},
  { 'name': 'hour', 'dataType': 'Int64'},
  { 'name': 'minutes', 'dataType': 'Int64'},
  { 'name': 'second', 'dataType': 'Int64'}
]
}
    
      

{'name': 'date',
 'columns': [{'name': 'time', 'dataType': 'time'},
  {'name': 'hour', 'dataType': 'Int64'},
  {'name': 'minutes', 'dataType': 'Int64'},
  {'name': 'second', 'dataType': 'Int64'}]}

# erp

<h2> agg interval age, card_tier_name,gender,province

In [35]:
client = Client(host = 'localhost',
                client_name = 'python-driver',
                port = 19000,
                user='primedata',
                password='primedata',
                secure=False,
                verify=False,
                database = 'eventify',
               )

In [36]:
def query_erp(start_time,end_time):
    query_transaction = f""" 
                select a.*,b.gender,c.Province,d.card_tier_name,e.dob from
                (
                select anonymous_id,at,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ngay_ct')] as ngay_ct,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.gio_gh')] as gio_gh,
                `num_properties.vals`[indexOf(`num_properties.keys`,'properties.gia_ban')] * `num_properties.vals`[indexOf(`num_properties.keys`,'properties.so_luong')] as order_value,
                `num_properties.vals`[indexOf(`num_properties.keys`,'properties.t_tien_sau_ck')] as t_tien_sau_ck,
                `num_properties.vals`[indexOf(`num_properties.keys`,'properties.so_luong')] as so_luong,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.stt_rec')] as stt_rec,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.dich_vu')] as dich_vu,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.dvt')] as dvt,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.nguon_dh')] as nguon_dh,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.status')] as status,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ma_vt')] as ma_vt,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ma_vt2')] as ma_vt2,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ma_ct_km')] as ma_ct_km,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ten_ct_km')] as ten_ct_km,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.nh_dvcs1')] as nh_dvcs1,
                `str_properties.vals`[indexOf(`str_properties.keys`,'properties.nh_dvcs3')] as nh_dvcs3 
                from events 
                where event_name = 'erp_item_order_completed' 
                and at >= toDateTime('{start_time}') AND at < toDateTime('{end_time}')
                ) a
                
                left outer join 
                
                (select anonymous_id, str_val as gender 
                from eventify_stag.profile_str_final_v 
                where str_key='gender') b
                on a.anonymous_id = b.anonymous_id
                
                left outer join 
                
                (select anonymous_id,str_val as Province
                from eventify_stag.profile_str_final_v 
                where str_key='city') c
                on a.anonymous_id = c.anonymous_id
                
                left outer join
                (select anonymous_id,str_val as card_tier_name 
                from eventify_stag.profile_str_final_v 
                where str_key='card_tier_name') d
                on a.anonymous_id = d.anonymous_id 
                
                left outer join
                (select anonymous_id,str_val as dob 
                from eventify_stag.profile_str_final_v 
                where str_key = 'dob') e 
                on a.anonymous_id = e.anonymous_id
                
            """
    iterator= client.execute_iter(query_transaction,{'max_block_size': 5000},with_column_types=True)
    columns = next(iterator)
    transaction_data=[]
    for item in iterator:
        transaction_data.append(item)
    transaction_df=pd.DataFrame(transaction_data[1:],columns=[col[0][2:] for col in columns])
    return transaction_df

In [37]:
def interval_age(age):
    if age in range(0,20):
        return '<20'
    if age in range(20,26):
        return '20-25'
    if age in range(26,31):
        return '26-30'
    if age in range(31,36):
        return '31-35'
    if age in range(36,41):
        return '36-40'
    if age in range(41,46):
        return '41-46'
    if age in range(46,51):
        return '46-50'
    if age in range(51,100):
        return ">50"
    return 'undefined'
    

In [38]:
def calculate_age(born):
    if born !="":
        today = date.today()
        return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [39]:
def convert_datetime(datetime_ob):
    if datetime_ob != "":
        dt = datetime.strptime(datetime_ob,"%Y-%m-%dT%H:%M:%S")
    else:
        dt = ""
    return dt

In [40]:
def agg_interval_age(dataframe):
    dataframe['age']= dataframe['dob'].apply(convert_datetime).apply(calculate_age)
    dataframe['interval_age'] = dataframe['age'].apply(interval_age)
    return dataframe 

In [41]:
def erp_pipeline(start_time,end_time):
    erp_query = query_erp(start_time,end_time)
    erp_query = agg_interval_age(erp_query)
    return erp_query
    
    

In [13]:
erp3

,anonymous_id,at,ngay_ct,gio_gh,order_value,t_tien_sau_ck,so_luong,stt_rec,dich_vu,dvt,...,ma_ct_km,ten_ct_km,nh_dvcs1,nh_dvcs3,gender,Province,card_tier_name,dob,age,interval_age
0,22vOFybZD9x7Lz9QnmaTYEAvuKA,2021-12-28 22:27:27,2021-12-28T00:00:00,22:27:27,750000.0,750000.0,1.0,K1000000004716350BL2,,CHIEC,...,,,SRMB,DQMB,Nữ,Hà Nội,,1988-05-26T00:00:00,33.0,31-35
1,22vOFybZD9x7Lz9QnmaTYEAvuKA,2021-12-28 22:27:27,2021-12-28T00:00:00,22:27:27,0.0,0.0,1.0,K1000000004716350BL2,,CHIEC,...,,,SRMB,DQMB,Nữ,Hà Nội,,1988-05-26T00:00:00,33.0,31-35
2,22vOFybZD9x7Lz9QnmaTYEAvuKA,2021-12-28 22:27:27,2021-12-28T00:00:00,22:27:27,0.0,0.0,1.0,K1000000004716350BL2,,CAI,...,,,SRMB,DQMB,Nữ,Hà Nội,,1988-05-26T00:00:00,33.0,31-35
3,1uchkxy2mSlSGWq2iSC4ry5KGmu,2021-12-30 12:18:38,2021-12-30T00:00:00,12:18:38,45000.0,45000.0,1.0,K1000000004764699BL2,,DOI,...,,,SRMT,DQMT,Nữ,Đà Nẵng,MEMBER,1900-01-01T00:00:00,122.0,undefined
4,1uchkxy2mSlSGWq2iSC4ry5KGmu,2021-12-30 12:18:38,2021-12-30T00:00:00,12:18:38,1250000.0,1250000.0,1.0,K1000000004764699BL2,,CHIEC,...,,,SRMT,DQMT,Nữ,Đà Nẵng,MEMBER,1900-01-01T00:00:00,122.0,undefined
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4698,2313udii0sapuqUqD5VNuCQftSv,2021-12-30 21:25:31,2021-12-30T00:00:00,21:25:31,325000.0,325000.0,1.0,K1000000004766163BL2,,CHIEC,...,,,SRMB,DQMB,,Ninh Bình,,,NaN,undefined
4699,2313uggMKVuWuFFY2MET1VPhtx3,2021-12-30 19:04:47,2021-12-30T00:00:00,19:04:47,110000.0,110000.0,2.0,K1000000004766012BL2,,DOI,...,,,SRMB,DQMB,,Hà Nội,,,NaN,undefined
4700,2313uhpJoSkxGgqtNPvpUuas4r7,2021-12-30 19:04:47,2021-12-30T00:00:00,19:04:47,570000.0,570000.0,1.0,K1000000004766012BL2,,CHIEC,...,,,SRMB,DQMB,,Hà Nội,,,NaN,undefined
4701,2313ukYqw7ztIITZhoXfo4iibSS,2021-12-30 21:25:31,2021-12-30T00:00:00,21:25:31,795000.0,795000.0,1.0,K1000000004766163BL2,,CHIEC,...,,,SRMB,DQMB,,Ninh Bình,,,NaN,undefined


In [28]:
from datetime import date, timedelta, datetime

def daterange(start_date, end_date):
    for n in range(0,int((end_date - start_date).days),7):
        yield start_date + timedelta(n)



    

In [22]:
start_date= datetime(2020,1,1,0,0,0)
end_date=datetime(2021,1,1,0,0,0)
query_result=[]
for single_date in daterange(start_date, end_date):
    batch_start = single_date.strftime("%Y-%m-%d %H:%M:%S")
    batch_end = (single_date + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S")
    transaction=query_erp(start_time = batch_start,end_time = batch_end)
    query_result.append(transaction)
    print(batch_start,batch_end,'status: Done \n','shape: ',transaction.shape)
    

2020-01-01 00:00:00 2020-01-08 00:00:00 status: Done 
 shape:  (10805, 22)
2020-01-08 00:00:00 2020-01-15 00:00:00 status: Done 
 shape:  (9318, 22)
2020-01-15 00:00:00 2020-01-22 00:00:00 status: Done 
 shape:  (28113, 22)
2020-01-22 00:00:00 2020-01-29 00:00:00 status: Done 
 shape:  (10843, 22)
2020-01-29 00:00:00 2020-02-05 00:00:00 status: Done 
 shape:  (4387, 22)
2020-02-05 00:00:00 2020-02-12 00:00:00 status: Done 
 shape:  (4188, 22)
2020-02-12 00:00:00 2020-02-19 00:00:00 status: Done 
 shape:  (5508, 22)
2020-02-19 00:00:00 2020-02-26 00:00:00 status: Done 
 shape:  (5687, 22)
2020-02-26 00:00:00 2020-03-04 00:00:00 status: Done 
 shape:  (8119, 22)
2020-03-04 00:00:00 2020-03-11 00:00:00 status: Done 
 shape:  (8618, 22)
2020-03-11 00:00:00 2020-03-18 00:00:00 status: Done 
 shape:  (9883, 22)
2020-03-18 00:00:00 2020-03-25 00:00:00 status: Done 
 shape:  (8089, 22)
2020-03-25 00:00:00 2020-04-01 00:00:00 status: Done 
 shape:  (6789, 22)
2020-04-01 00:00:00 2020-04-08 00:0

In [531]:
query_result[-1]['at'].max()

Timestamp('2021-01-05 22:25:06')

In [554]:
start_date= datetime(2021,1,6,0,0,0)
end_date=datetime(2021,12,29,0,0,0)
query_result2=[]
for single_date in daterange(start_date, end_date):
    batch_start = single_date.strftime("%Y-%m-%d %H:%M:%S")
    batch_end = (single_date + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S")
    transaction=query_erp(start_time = batch_start,end_time = batch_end)
    query_result2.append(transaction)
    print(batch_start,batch_end,'status: Done \n','shape: ',transaction.shape)
    

2021-01-06 00:00:00 2021-01-13 00:00:00 status: Done 
 shape:  (13531, 22)
2021-01-13 00:00:00 2021-01-20 00:00:00 status: Done 
 shape:  (11778, 22)
2021-01-20 00:00:00 2021-01-27 00:00:00 status: Done 
 shape:  (17203, 22)
2021-01-27 00:00:00 2021-02-03 00:00:00 status: Done 
 shape:  (20579, 22)
2021-02-03 00:00:00 2021-02-10 00:00:00 status: Done 
 shape:  (29781, 22)
2021-02-10 00:00:00 2021-02-17 00:00:00 status: Done 
 shape:  (5982, 22)
2021-02-17 00:00:00 2021-02-24 00:00:00 status: Done 
 shape:  (6073, 22)
2021-02-24 00:00:00 2021-03-03 00:00:00 status: Done 
 shape:  (6977, 22)
2021-03-03 00:00:00 2021-03-10 00:00:00 status: Done 
 shape:  (10275, 22)
2021-03-10 00:00:00 2021-03-17 00:00:00 status: Done 
 shape:  (15025, 22)
2021-03-17 00:00:00 2021-03-24 00:00:00 status: Done 
 shape:  (14074, 22)
2021-03-24 00:00:00 2021-03-31 00:00:00 status: Done 
 shape:  (15166, 22)
2021-03-31 00:00:00 2021-04-07 00:00:00 status: Done 
 shape:  (15827, 22)
2021-04-07 00:00:00 2021-04-

In [23]:
erp1 = pd.concat(query_result).reset_index(drop=True)

In [24]:
erp1 = agg_interval_age(erp1)

In [557]:
len(erp1)

699426

In [558]:
erp2=pd.concat(query_result2).reset_index(drop=True)

In [559]:
erp2 = agg_interval_age(erp2)

In [29]:
def merge_product_and_erp(erp_data):
    product = pd.read_csv(r'C:\Users\ADMIN\Desktop\hello\Sale dashboard\product.txt',sep='\t',encoding='utf-8')
    erp_data=erp_data.merge(product[['ma_vt','nh_vt2','nh_vt5','nh_vt8','nh_vt10','nh_vt12','nh_vt13','nh_vt14']],on='ma_vt',how='left')
    return erp_data

In [30]:
def erp_pipeline(start_time, end_time):
    erp_query = query_erp(start_time, end_time)
    erp_result = agg_interval_age(erp_query)
    erp_result['Province'] = erp_result['Province'].apply(replace_EmptyNanNull)
    erp_result['card_tier_name'] = erp_result['card_tier_name'].apply(replace_EmptyNanNull)
    erp_result['interval_age']= erp_result['interval_age'].apply(replace_EmptyNanNull)
    erp_result['gender']=erp_result['gender'].apply(replace_EmptyNanNull)
    return erp_result

In [31]:
def replace_EmptyNanNull(x):
    if x == None or x == '' or x==np.NaN:
        return 'undefined'
    else:
        return x

In [560]:
erp1 = merge_product_and_erp(erp1)
erp2 = merge_product_and_erp(erp2)

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_24052/4145699638.py:1: DtypeWarning: Columns (1,3,4,5,7,8,10,11,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,52,53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  erp1 = merge_product_and_erp(erp1)
C:\Users\ADMIN\AppData\Local\Temp/ipykernel_24052/4145699638.py:2: DtypeWarning: Columns (1,3,4,5,7,8,10,11,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,52,53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  erp2 = merge_product_and_erp(erp2)


In [42]:
erp3 = erp_pipeline("2021-12-28 22:27:27","2022-01-02 12:00:00")

erp3 = merge_product_and_erp(erp3)

In [17]:
erp3.to_csv('erp3.txt',mode = 'w', sep = ';', header = True, index = None,encoding = 'utf-8')

In [19]:
erp3.sort_values('ngay_ct',ascending=False)

,anonymous_id,at,ngay_ct,gio_gh,order_value,t_tien_sau_ck,so_luong,stt_rec,dich_vu,dvt,...,dob,age,interval_age,nh_vt2,nh_vt5,nh_vt8,nh_vt10,nh_vt12,nh_vt13,nh_vt14
0,23CLHKIOTeM4St7sbr0Tr9OtiNS,2022-01-03 12:24:35,2022-01-03T00:00:00,12:24:35,1250000.0,1250000.0,1.0,K1000000004772968BL2,,CHIEC,...,1985-09-30T00:00:00,36.0,36-40,N02-ARISTINO,N05-2021,N08-JACKET,N10-E00200,N12-00,N13-03,N14-PE
7193,1uqV2PWD4GP9lou1ZSHO3D6PhQr,2022-01-03 15:46:58,2022-01-03T00:00:00,15:46:58,550000.0,522500.0,1.0,K1000000004773443BL2,,CHIEC,...,1900-01-01T00:00:00,122.0,undefined,N02-ARISTINO,N05-2021,N08-QUANTHUDONG,N10-N00800,N12-00,N13-03,N14-CV
7199,1uqVP8zQeqinkT7IKoeZXQVymno,2022-01-03 21:49:33,2022-01-03T00:00:00,21:49:33,-850000.0,-807500.0,-1.0,K1000000004774133BL2,,CHIEC,...,1976-11-09T00:00:00,45.0,41-46,N02-ARISTINO,N05-2020,N08-QUANAU,N10-E09507,N12-07,N13-06,N14-PE
7198,1uqVP8zQeqinkT7IKoeZXQVymno,2022-01-03 21:49:33,2022-01-03T00:00:00,21:49:33,55000.0,52250.0,1.0,K1000000004774133BL2,,DOI,...,1976-11-09T00:00:00,45.0,41-46,N02-BIZMEN,N05-2019,N08-TATNAM,N10-G01803,N12-03,N13-34,N14-MC
7197,1uqVP8zQeqinkT7IKoeZXQVymno,2022-01-03 21:49:33,2022-01-03T00:00:00,21:49:33,750000.0,712500.0,1.0,K1000000004774133BL2,,CHIEC,...,1976-11-09T00:00:00,45.0,41-46,N02-ARISTINO,N05-2020,N08-QUANAU,N10-G11800,N12-00,N13-07,N14-TR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224,1uddthQm3Enoe9U6wqZJe65gecz,2021-12-29 20:04:12,2021-12-29T00:00:00,20:04:12,950000.0,902500.0,1.0,K1000000004740321BL2,,CHIEC,...,1900-01-01T00:00:00,122.0,undefined,N02-ARISTINO,N05-2021,N08-QUANAU,N10-N02500,N12-00,N13-02,N14-TR
50,22vOFybZD9x7Lz9QnmaTYEAvuKA,2021-12-28 22:27:27,2021-12-28T00:00:00,22:27:27,750000.0,750000.0,1.0,K1000000004716350BL2,,CHIEC,...,1988-05-26T00:00:00,33.0,31-35,N02-ARISTINO,N05-2021,N08-QUANAU,N10-B02507,N12-07,N13-02,N14-TR
51,22vOFybZD9x7Lz9QnmaTYEAvuKA,2021-12-28 22:27:27,2021-12-28T00:00:00,22:27:27,0.0,0.0,1.0,K1000000004716350BL2,,CHIEC,...,1988-05-26T00:00:00,33.0,31-35,N02-NONE,N05-NONE,N08-NPL.NONE,N10-NONE,NaN,NaN,NaN
52,22vOFybZD9x7Lz9QnmaTYEAvuKA,2021-12-28 22:27:27,2021-12-28T00:00:00,22:27:27,0.0,0.0,1.0,K1000000004716350BL2,,CAI,...,1988-05-26T00:00:00,33.0,31-35,N02-NONE,N05-NONE,N08-CCDC,N10-NONE,NaN,NaN,NaN


In [581]:
erp1['Province'].replace('','undefined',inplace=True)

erp1['card_tier_name'].replace('','undefined',inplace=True)

erp1['interval_age'].replace('','undefined',inplace=True)
erp1['gender'].replace('','undefined',inplace=True)

In [580]:
erp2['Province'].replace('','undefined',inplace=True)

erp2['card_tier_name'].replace('','undefined',inplace=True)

erp2['interval_age'].replace('','undefined',inplace=True)
erp2['gender'].replace('','undefined',inplace=True)

In [575]:
erp2[erp2['interval_age']=='']

,anonymous_id,at,ngay_ct,gio_gh,order_value,t_tien_sau_ck,so_luong,stt_rec,dich_vu,dvt,...,dob,age,interval_age,nh_vt2,nh_vt5,nh_vt8,nh_vt10,nh_vt12,nh_vt13,nh_vt14


In [582]:
erp1.to_csv('erp1.txt',mode = 'w', sep = ';', header = True, index = None,encoding = 'utf-8')
erp2.to_csv('erp2.txt',mode = 'w', sep = ';', header = True, index = None,encoding = 'utf-8')


In [25]:
def transform_and_jsondump(data_push):
    for i in data_push:
        if data_push[i].dtypes == 'datetime64[ns]':
            data_push[i] = data_push[i].astype(str)
    data_push = data_push.to_dict('records')
    data_push = json.dumps(data_push)
    return data_push

In [337]:
for i in erp:
    if erp[i].dtypes == 'datetime64[ns]':
        erp[i] = erp[i].astype(str)

In [ ]:
{'name': 'date', 'columns':
 [{ 'name': 'time', 'dataType': 'time'},
  { 'name': 'hour', 'dataType': 'Int64'},
  { 'name': 'minutes', 'dataType': 'Int64'},
  { 'name': 'second', 'dataType': 'Int64'}
]
  }
    
      

In [420]:
def show_table(dataframe):
    erp_dtypes = dataframe.dtypes.to_frame(name ='dtypes').reset_index()
    list_col = []
    for i in range(len(erp_dtypes)):
        if erp_dtypes.iloc[i]['dtypes'] == 'object':
            list_col.append({'name':erp_dtypes.iloc[i]['index'],'dataType':'string'})
        if erp_dtypes.iloc[i]['dtypes'] == 'float':
            list_col.append({'name':erp_dtypes.iloc[i]['index'],'dataType':'decimal'})
    return list_col
      
     
    

In [421]:
list_col

[{'name': 'anonymous_id', 'dataType': 'string'},
 {'name': 'at', 'dataType': 'string'},
 {'name': 'order_value', 'dataType': 'decimal'},
 {'name': 't_tien_sau_ck', 'dataType': 'decimal'},
 {'name': 'so_luong', 'dataType': 'decimal'},
 {'name': 'stt_rec', 'dataType': 'string'},
 {'name': 'dich_vu', 'dataType': 'string'},
 {'name': 'dvt', 'dataType': 'string'},
 {'name': 'nguon_dh', 'dataType': 'string'},
 {'name': 'status', 'dataType': 'string'},
 {'name': 'ma_vt', 'dataType': 'string'},
 {'name': 'ma_vt2', 'dataType': 'string'},
 {'name': 'ma_ct_km', 'dataType': 'string'},
 {'name': 'ten_ct_km', 'dataType': 'string'},
 {'name': 'nh_dvcs1', 'dataType': 'string'},
 {'name': 'nh_dvcs3', 'dataType': 'string'},
 {'name': 'gender', 'dataType': 'string'},
 {'name': 'Province', 'dataType': 'string'},
 {'name': 'card_tier_name', 'dataType': 'string'},
 {'name': 'dob', 'dataType': 'string'},
 {'name': 'age', 'dataType': 'decimal'},
 {'name': 'interval_age', 'dataType': 'string'}]

In [428]:
create_table_erp={'name': 'erp', 'columns':list_col}


In [429]:
create_table_erp

{'name': 'erp',
 'columns': [{'name': 'anonymous_id', 'dataType': 'string'},
  {'name': 'at', 'dataType': 'string'},
  {'name': 'order_value', 'dataType': 'decimal'},
  {'name': 't_tien_sau_ck', 'dataType': 'decimal'},
  {'name': 'so_luong', 'dataType': 'decimal'},
  {'name': 'stt_rec', 'dataType': 'string'},
  {'name': 'dich_vu', 'dataType': 'string'},
  {'name': 'dvt', 'dataType': 'string'},
  {'name': 'nguon_dh', 'dataType': 'string'},
  {'name': 'status', 'dataType': 'string'},
  {'name': 'ma_vt', 'dataType': 'string'},
  {'name': 'ma_vt2', 'dataType': 'string'},
  {'name': 'ma_ct_km', 'dataType': 'string'},
  {'name': 'ten_ct_km', 'dataType': 'string'},
  {'name': 'nh_dvcs1', 'dataType': 'string'},
  {'name': 'nh_dvcs3', 'dataType': 'string'},
  {'name': 'gender', 'dataType': 'string'},
  {'name': 'Province', 'dataType': 'string'},
  {'name': 'card_tier_name', 'dataType': 'string'},
  {'name': 'dob', 'dataType': 'string'},
  {'name': 'age', 'dataType': 'decimal'},
  {'name': 'inte

<h2> transform table product

get product newAPi

In [109]:
import requests

In [110]:
def daterange_api(start_date, end_date):
    for n in range(0,int((end_date - start_date).days),180):
        yield start_date + timedelta(n)

In [111]:
def get_product_new(start_date,end_date,headers):
    payload = {'ngay_bd': start_date,'ngay_kt': end_date}
    r = requests.get("https://crm.aristino.com:4436/KGAPI/api/CategoryAPI/GetProducts_ByUpdate",headers={'key': headers},
                     params = payload)
    return r

In [112]:
to

In [113]:
start_date = date(2015,1,1)
end_date = date.today()
key='58cf21a9-af57-4fdd-8234-034e05aedfc2'
resp = get_product_new(start_date,end_date,key)
productNewapi = read_resp(resp)

    

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_24304/4006870286.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.io.json.json_normalize(df.Result)


,ma_vt,ten_vt,ma_vt2,ten_vt2,dvt,nh_vt1,nh_vt2,nh_vt3,nh_vt4,nh_vt5,...,ten_nh_vt14,ten_nh_vt15,ten_nh_vt16,ten_nh_vt17,ten_nh_vt18,ten_nh_vt19,ten_nh_vt20,ten_nh_vt21,ghi_chu,status
0,4KKMBX02.2,quần sịp boxer chun Kwin KKMBX02 cỡ S,4KKMBX02,quần sịp boxer chun Kwin KKMBX02,CHIEC,KKMBX02,N02-KWINMEN,N03-SIP-BOXER,N04-0,N05-2019,...,,,,,,Sịp Boxer,,,,1
1,3152508XXXX.2,BTP -Khăn Salina SD02 cỡ S,3152508XXXX,BTP -Khăn Salina SD02,CHIEC,BTP-SD02,N02-SALINA,N03-KHANMAT,N04-0,N05-NONE,...,Bamboo,,,,,Khăn,,,,1
2,ASH06201KPE42K000104,Giày thể thao nam ASH06201 cỡ 40,ASH06201KPE42K00010,Giày thể thao nam ASH06201,DOI,ASH06201,N02-ARISTINO,N03-GIAY,N04-0,N05-2021,...,Polyester / Micro PES/ Microfiber,,,,,,,,,1
3,ASH08001LLT43K000363,Giày thể thao nam ASH08001 cỡ 39,ASH08001LLT43K00036,Giày thể thao nam ASH08001,DOI,ASH08001,N02-ARISTINO,N03-GIAY,N04-0,N05-2021,...,Leather,,,,,,,,,1
4,BSC00700KCO00K009000,Bít tất BSC007 Đen 9,BSC00700KCO00K00900,Bít tất BSC007 Đen 9,DOI,BSC007,N02-BIZMEN,N03-TATCONGSO,N04-0,N05-2018,...,Cotton,,,,,Tất,,,,1


get product old APi

In [115]:
import psycopg2

In [116]:
conn = psycopg2.connect(
    host="localhost",
    user="pd",
    password="iloveprimedata",
    dbname='eventify_stag',
    port=45432)

In [117]:
cur = conn.cursor()

In [118]:
query_product = pd.read_sql_query("""select * from public.product_item;""",conn)

In [119]:
product1 = query_product[['id']].copy()
product1.rename(columns={'id':'ma_vt'},inplace=True) # rename id to ma_vt
#convert columns properties with json type to table
product2= pd.io.json.json_normalize(query_product.properties)[['nh_vt2','nh_vt5','nh_vt8','nh_vt10','nh_vt12','nh_vt13','nh_vt14']]
product= pd.concat([product1,product2],axis=1)

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_24304/3644626167.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  product2= pd.io.json.json_normalize(query_product.properties)[['nh_vt2','nh_vt5','nh_vt8','nh_vt10','nh_vt12','nh_vt13','nh_vt14']]


merge product new API voiws product postgres

In [597]:
productNewapi.columns

Index(['ma_vt', 'ten_vt', 'ma_vt2', 'ten_vt2', 'dvt', 'nh_vt1', 'nh_vt2',
       'nh_vt3', 'nh_vt4', 'nh_vt5', 'nh_vt6', 'nh_vt7', 'nh_vt8', 'nh_vt9',
       'nh_vt10', 'nh_vt11', 'nh_vt12', 'nh_vt13', 'nh_vt14', 'nh_vt15',
       'nh_vt16', 'nh_vt17', 'nh_vt18', 'nh_vt19', 'nh_vt20', 'nh_vt21',
       'ten_nh_vt1', 'ten_nh_vt2', 'ten_nh_vt3', 'ten_nh_vt4', 'ten_nh_vt5',
       'ten_nh_vt6', 'ten_nh_vt7', 'ten_nh_vt8', 'ten_nh_vt9', 'ten_nh_vt10',
       'ten_nh_vt11', 'ten_nh_vt12', 'ten_nh_vt13', 'ten_nh_vt14',
       'ten_nh_vt15', 'ten_nh_vt16', 'ten_nh_vt17', 'ten_nh_vt18',
       'ten_nh_vt19', 'ten_nh_vt20', 'ten_nh_vt21', 'ghi_chu', 'status'],
      dtype='object')

In [608]:
product.columns

Index(['ma_vt', 'nh_vt2', 'nh_vt5', 'nh_vt8', 'nh_vt10', 'nh_vt12', 'nh_vt13',
       'nh_vt14'],
      dtype='object')

In [32]:
# merge erp to get nh_Vt
erp1 = erp1.merge(product,on='ma_vt',how='left')

In [33]:
erp1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 631704 entries, 0 to 631703
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   anonymous_id    631704 non-null  object        
 1   at              631704 non-null  datetime64[ns]
 2   ngay_ct         631704 non-null  object        
 3   gio_gh          631704 non-null  object        
 4   order_value     631704 non-null  float64       
 5   t_tien_sau_ck   631704 non-null  float64       
 6   so_luong        631704 non-null  float64       
 7   stt_rec         631704 non-null  object        
 8   dich_vu         631704 non-null  object        
 9   dvt             631704 non-null  object        
 10  nguon_dh        631704 non-null  object        
 11  status          631704 non-null  object        
 12  ma_vt           631704 non-null  object        
 13  ma_vt2          631704 non-null  object        
 14  ma_ct_km        631704 non-null  obj

In [603]:
#product.merge(productNewapi[['ma_vt','ten_nh_vt2','ten_nh_vt5','ten_nh_vt8','ten_nh_vt10','ten_nh_vt12','ten_nh_vt13','ten_nh_vt14']],on='ma_vt',how='left')

In [ ]:
import numpy as np
def replace_EmptyNanNull(x):
    if x == None or x == '' or x==np.NaN:
        return 'undefined'
    else:
        return x

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

N13-02    35390
N13-03    30724
N13-00    25060
          11931
N13-10     5260
N13-06     3326
N13-33     1593
N13-12     1372
N13-07     1097
N13-39      773
N13-38      616
N13-13      219
N13-20      170
N13-27      132
N13-23      131
N13-26       67
N13-22       65
N13-28       36
N13-34       28
N13-05        2
Name: nh_vt13, dtype: int64

In [84]:
erp1['nh_vt13'].apply(replace_EmptyNanNull).value_counts()

N13-02       35390
N13-03       30724
N13-00       25060
undefined    11931
N13-10        5260
N13-06        3326
N13-33        1593
N13-12        1372
N13-07        1097
N13-39         773
N13-38         616
N13-13         219
N13-20         170
N13-27         132
N13-23         131
N13-26          67
N13-22          65
N13-28          36
N13-34          28
N13-05           2
Name: nh_vt13, dtype: int64

In [73]:
erp1.nh_vt12

0         None
1         None
2         None
3         None
4         None
          ... 
631699    None
631700    None
631701    None
631702    None
631703    None
Name: nh_vt12, Length: 631704, dtype: object

In [229]:
def merge_product_and_erp(erp,product):
    erp=erp.merge(product[['ma_vt','nh_vt2','nh_vt5','nh_vt8','nh_vt10','nh_vt12','nh_vt13','nh_vt14']],on='ma_vt',how='left')
    return erp

In [427]:
productNewapi_dtypes = productNewapi.dtypes.to_frame(name ='dtypes').reset_index()
list_col_product = []
for i in range(len(productNewapi_dtypes)):
    if productNewapi_dtypes.iloc[i]['dtypes'] == 'object':
        list_col_product.append({'name':productNewapi_dtypes.iloc[i]['index'],'dataType':'string'})
    if productNewapi_dtypes.iloc[i]['dtypes'] == 'float':
        list_col_product.append({'name':productNewapi_dtypes.iloc[i]['index'],'dataType':'decimal'})
      
     
    

In [474]:
create_table_product={
      'name': 'segment',
      'tables': [{'name': 'product', 'columns':list_col_product},
                {'name': 'erp', 'columns':list_col}],
        'relationship'
}

In [451]:
product = json.loads(resp.text)['Result']

In [473]:
def create_dataset(data):
    data = json.dumps(data)
    return s.post('https://api.powerbi.com/v1.0/myorg/datasets',data)

In [475]:
createdataset = create_dataset(create_table_product)

In [476]:
createdataset

<Response [201]>

In [490]:
#add row into dataset
def add_row_into_dataset(datasetId,tableName,data):
    request_url = f'https://api.powerbi.com/v1.0/myorg/datasets/{datasetId}/tables/{tableName}/rows'
    resp = s.post(request_url,data)
    if resp.status_code != 200:
        raise Exception(f"Failed to push data to PowerBI: [{resp.status_code}] - {resp.text}")
    return resp
                  

In [477]:
datasetId='0f35f863-5c9c-4754-aa99-e4ef8198ce92'
tableName='product'

In [478]:
count = 0
while count < len(product):
    data = product[count:count+9900]
    post_row = add_row_into_dataset(datasetId,tableName,data)
    if post_row.status_code != 200:
        raise Exception(f"Failed at batch ...",f"{post_row.text}",f'batch_index {batch}')
    count+= 9900

In [493]:
erp['at']

0        2020-02-16 17:38:08
1        2020-02-16 17:38:08
2        2020-02-16 17:38:08
3        2020-02-16 17:38:08
4        2020-02-16 14:42:33
                 ...        
251301   2020-01-18 16:47:21
251302   2020-01-18 16:47:21
251303   2020-01-18 16:48:30
251304   2020-01-18 16:48:30
251305   2020-01-18 16:48:30
Name: at, Length: 251306, dtype: datetime64[ns]

In [505]:
count = 0
datasetId='0f35f863-5c9c-4754-aa99-e4ef8198ce92'
tableName='erp'
while count < len(erp):
    data = erp.iloc[count:count+9900]
    data_push = transform_and_jsondump(data)
    post_row = add_row_into_dataset(datasetId,tableName,data_push)
    if post_row.status_code != 200:
        raise Exception(f"Failed at batch ...",f"{post_row.text}",f'batch_index {batch}')
    count+= 9900

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_24052/2244525372.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_push[i] = data_push[i].astype(str)


In [128]:
product_new = pd.concat([productNewapi,product], axis=0, ignore_index=True)

In [135]:
def get_unique_field(product,field1,field2):
    result = product[[field1,field2]].groupby([field1,field2]).size().reset_index(name='Freq')
    return result.drop(columns='Freq')
    


In [137]:
get_unique_field(product_new,'nh_vt2','ten_nh_vt2')

,nh_vt2,ten_nh_vt2
0,N02-ARISTINO,Aristino
1,N02-ARISTINOBOSS,Aristino Boss
2,N02-BIOZONE,Biozone
3,N02-BIZMEN,Bizmen
4,N02-BIZS+,BIZS+
5,N02-CECINA,Cecina
6,N02-INSIDEMEN,Insidemen
7,N02-KIDMAN,Kidman
8,N02-KWINMEN,Kwin
9,N02-NONE,Không phân thương hiệu


In [136]:
product_new[['nh_vt2','ten_nh_vt2']].groupby(['nh_vt2','ten_nh_vt2']).size().reset_index(name='Freq')

,nh_vt2,ten_nh_vt2,Freq
0,N02-ARISTINO,Aristino,30882
1,N02-ARISTINOBOSS,Aristino Boss,31
2,N02-BIOZONE,Biozone,20
3,N02-BIZMEN,Bizmen,464
4,N02-BIZS+,BIZS+,137
5,N02-CECINA,Cecina,1483
6,N02-INSIDEMEN,Insidemen,4782
7,N02-KIDMAN,Kidman,449
8,N02-KWINMEN,Kwin,4123
9,N02-NONE,Không phân thương hiệu,4643


# test delete row in dataset powerBi

In [138]:
import msal
import requests
import json

# --------------------------------------------------
# Set local variables
# --------------------------------------------------
client_id='b1e937d4-ade0-4aec-ba94-529a143c9cc1'
username = 'integration@primedata.ai'
password = 'jTbCm.3RR49RqARv'
authority_url = 'https://login.microsoftonline.com/primedata.ai'
scope = ["https://analysis.windows.net/powerbi/api/.default"]
url_groups = 'https://api.powerbi.com/v1.0/myorg/groups'

# --------------------------------------------------
# Use MSAL to grab a token
# --------------------------------------------------
app = msal.PublicClientApplication(client_id, authority=authority_url)
result = app.acquire_token_by_username_password(username=username,password=password,scopes=scope)

# --------------------------------------------------
# Check if a token was obtained, grab it and call the
# Power BI REST API, otherwise throw up the error message
# --------------------------------------------------
if 'access_token' in result:
    access_token = result['access_token']
    header = {'Content-Type':'application/json','Authorization': f'Bearer {access_token}'}
    api_out = requests.get(url=url_groups, headers=header)
    print(api_out.json())
else:
    print(result.get("error"))
    print(result.get("error_description"))

{'@odata.context': 'http://wabi-south-east-asia-b-primary-redirect.analysis.windows.net/v1.0/myorg/$metadata#groups', '@odata.count': 2, 'value': [{'id': '7da7604f-20b3-49c0-aee5-6eb0c2cc010d', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'Default'}, {'id': '66871624-e0e7-4314-92ca-24e2864597b6', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'aristino'}]}


In [139]:
s = requests.Session()

#s.headers['Content-Type'] = 'application/json'
s.headers['Authorization'] = f'Bearer {access_token}'

In [166]:
def delete_row(datasetId,tableName):
    return s.delete(f'https://api.powerbi.com/v1.0/myorg/datasets/{datasetId}/tables/{tableName}/rows')

In [167]:
datasetId='0f35f863-5c9c-4754-aa99-e4ef8198ce92'
tableName='erp'


In [168]:
delete = delete_row(datasetId,tableName)

In [169]:
delete

<Response [200]>

In [170]:
json.loads(delete.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# create relationship in dataset powerBi

In [8]:
#date dimension
date_columns =[
  { 'name': 'date', 'dataType': 'DateTime','formatString':"dd/mm/yyyy"},
  { 'name': 'week_day', 'dataType': 'string'},
  { 'name': 'day', 'dataType': 'Int64'},
  { 'name': 'month', 'dataType': 'Int64'},
  { 'name': 'year', 'dataType': 'Int64'},
  { 'name': 'week', 'dataType': 'Int64'},
  { 'name': 'quarter', 'dataType': 'Int64'}
]

    
      

In [9]:
#Time dimension
time_columns = [{ 'name': 'Time', 'dataType': 'DateTime','formatString': "h:nn:ss"},
  { 'name': 'hour', 'dataType': 'Int64'},
  { 'name': 'minutes', 'dataType': 'Int64'},
  { 'name': 'second', 'dataType': 'Int64'}
]
    
      

In [124]:
def get_TypeOfColumns(dataframe): #convert type of columns to map with powerbi
    erp_dtypes = dataframe.dtypes.to_frame(name ='dtypes').reset_index()
    list_col = []
    for i in range(len(erp_dtypes)):
        if erp_dtypes.iloc[i]['dtypes'] == 'object':
            list_col.append({'name':erp_dtypes.iloc[i]['index'],'dataType':'string'})
        if erp_dtypes.iloc[i]['dtypes'] == 'float':
            list_col.append({'name':erp_dtypes.iloc[i]['index'],'dataType':'decimal'})
        if erp_dtypes.iloc[i]['dtypes'] == 'int':
            list_col.append({'name':erp_dtypes.iloc[i]['index'],'dataType':'Int64'})
    return list_col

In [45]:
get_TypeOfColumns(erp3)

[{'name': 'anonymous_id', 'dataType': 'string'},
 {'name': 'at', 'dataType': 'string'},
 {'name': 'ngay_ct', 'dataType': 'string'},
 {'name': 'gio_gh', 'dataType': 'string'},
 {'name': 'order_value', 'dataType': 'string'},
 {'name': 't_tien_sau_ck', 'dataType': 'string'},
 {'name': 'so_luong', 'dataType': 'string'},
 {'name': 'stt_rec', 'dataType': 'string'},
 {'name': 'dich_vu', 'dataType': 'string'},
 {'name': 'dvt', 'dataType': 'string'},
 {'name': 'nguon_dh', 'dataType': 'string'},
 {'name': 'status', 'dataType': 'string'},
 {'name': 'ma_vt2', 'dataType': 'string'},
 {'name': 'ma_ct_km', 'dataType': 'string'},
 {'name': 'ten_ct_km', 'dataType': 'string'},
 {'name': 'nh_dvcs1', 'dataType': 'string'},
 {'name': 'nh_dvcs3', 'dataType': 'string'},
 {'name': 'gender', 'dataType': 'string'},
 {'name': 'Province', 'dataType': 'string'},
 {'name': 'card_tier_name', 'dataType': 'string'},
 {'name': 'dob', 'dataType': 'string'},
 {'name': 'age', 'dataType': 'string'},
 {'name': 'interval_age

In [10]:
#erp columns
erp_columns = [{'name': 'anonymous_id', 'dataType': 'string'},
  {'name': 'ngay_ct', 'dataType': 'Datetime','formatString':"dd/mm/yyyy"},
  {'name': 'gio_gh', 'dataType': 'Datetime','formatString': "h:nn:ss"},
  {'name': 'order_value', 'dataType': 'decimal'},
  {'name': 't_tien_sau_ck', 'dataType': 'decimal'},
  {'name': 'so_luong', 'dataType': 'decimal'},
  {'name': 'stt_rec', 'dataType': 'string'},
  {'name': 'dich_vu', 'dataType': 'string'},
  {'name': 'dvt', 'dataType': 'string'},
  {'name': 'nguon_dh', 'dataType': 'string'},
  {'name': 'status', 'dataType': 'string'},
  {'name': 'ma_vt', 'dataType': 'string'},
  {'name': 'ma_vt2', 'dataType': 'string'},
  {'name': 'ma_ct_km', 'dataType': 'string'},
  {'name': 'ten_ct_km', 'dataType': 'string'},
  {'name': 'nh_dvcs1', 'dataType': 'string'},
  {'name': 'nh_dvcs3', 'dataType': 'string'},
  {'name': 'gender', 'dataType': 'string'},
  {'name': 'Province', 'dataType': 'string'},
  {'name': 'card_tier_name', 'dataType': 'string'},
  {'name': 'interval_age', 'dataType': 'string'},
  {'name': 'nh_vt2', 'dataType': 'string'},
  {'name': 'nh_vt5', 'dataType': 'string'},
  {'name': 'nh_vt8', 'dataType': 'string'},
  {'name': 'nh_vt10', 'dataType': 'string'},
  {'name': 'nh_vt12', 'dataType': 'string'},
  {'name': 'nh_vt13', 'dataType': 'string'},
  {'name': 'nh_vt14', 'dataType': 'string'}]

In [43]:
#get columns product in postges
import pandas as pd

In [11]:
product_columns=[{'name': 'ma_vt', 'dataType': 'string'},
  {'name': 'ma_vt2', 'dataType': 'string'},
  {'name': 'ten_vt', 'dataType': 'string'},
  {'name': 'ten_vt2', 'dataType': 'string'},
  {'name': 'dvt', 'dataType': 'string'},
  {'name': 'nh_vt1', 'dataType': 'string'},
  {'name': 'nh_vt2', 'dataType': 'string'},
  {'name': 'nh_vt3', 'dataType': 'string'},
  {'name': 'nh_vt4', 'dataType': 'string'},
  {'name': 'nh_vt5', 'dataType': 'string'},
  {'name': 'nh_vt6', 'dataType': 'string'},
  {'name': 'nh_vt7', 'dataType': 'string'},
  {'name': 'nh_vt8', 'dataType': 'string'},
  {'name': 'nh_vt9', 'dataType': 'string'},
  {'name': 'nh_vt10', 'dataType': 'string'},
  {'name': 'nh_vt11', 'dataType': 'string'},
  {'name': 'nh_vt12', 'dataType': 'string'},
  {'name': 'nh_vt13', 'dataType': 'string'},
  {'name': 'nh_vt14', 'dataType': 'string'},
  {'name': 'nh_vt15', 'dataType': 'string'},
  {'name': 'nh_vt16', 'dataType': 'string'},
  {'name': 'nh_vt17', 'dataType': 'string'},
  {'name': 'nh_vt18', 'dataType': 'string'},
  {'name': 'nh_vt19', 'dataType': 'string'},
  {'name': 'nh_vt20', 'dataType': 'string'},
  {'name': 'nh_vt21', 'dataType': 'string'},
  {'name': 'ten_nh_vt1', 'dataType': 'string'},
  {'name': 'ten_nh_vt2', 'dataType': 'string'},
  {'name': 'ten_nh_vt3', 'dataType': 'string'},
  {'name': 'ten_nh_vt4', 'dataType': 'string'},
  {'name': 'ten_nh_vt5', 'dataType': 'string'},
  {'name': 'ten_nh_vt6', 'dataType': 'string'},
  {'name': 'ten_nh_vt7', 'dataType': 'string'},
  {'name': 'ten_nh_vt8', 'dataType': 'string'},
  {'name': 'ten_nh_vt9', 'dataType': 'string'},
  {'name': 'ten_nh_vt10', 'dataType': 'string'},
  {'name': 'ten_nh_vt11', 'dataType': 'string'},
  {'name': 'ten_nh_vt12', 'dataType': 'string'},
  {'name': 'ten_nh_vt13', 'dataType': 'string'},
  {'name': 'ten_nh_vt14', 'dataType': 'string'},
  {'name': 'ten_nh_vt15', 'dataType': 'string'},
  {'name': 'ten_nh_vt16', 'dataType': 'string'},
  {'name': 'ten_nh_vt17', 'dataType': 'string'},
  {'name': 'ten_nh_vt18', 'dataType': 'string'},
  {'name': 'ten_nh_vt19', 'dataType': 'string'},
  {'name': 'ten_nh_vt20', 'dataType': 'string'},
  {'name': 'ten_nh_vt21', 'dataType': 'string'},
  {'name': 'ghi_chu', 'dataType': 'string'},
  {'name': 'status', 'dataType': 'string'}]

In [12]:
#user
{'name': 'user', 'columns':[{'name': 'user_id','dataType':'string'}]}

{'name': 'user', 'columns': [{'name': 'user_id', 'dataType': 'string'}]}

In [13]:
fact_segment_columns = [
    {'name':'user_id','dataType':'string'},
    {'name':'segment_transform','dataType':'string'}
]

In [14]:
dim_segment_columns=[
    {'name':'segment_id','dataType':'string'},
    {'name':'name','dataType':'string'},
    {'name':'segment_transform','dataType':'string'},
    {'name':'at','dataType':'DateTime'},
]

In [15]:
#Create Json data include Dataset_name, table name, columns name and columns types
result={
      'name': 'dev_test_segment_dashboard',
      'tables': [
          {'name': 'date', 'columns':date_columns},
          {'name': 'time', 'columns':time_columns},
          {'name': 'erp', 'columns':erp_columns},
          {'name': 'product', 'columns': product_columns},
          {'name': 'user', 'columns':[{'name': 'user_id','dataType':'string'}]},
          {'name': 'segment', 'columns':segment_columns},
          {'name': 'dim_segment', 'columns':dim_segment_columns}
      ]
      }


NameError: name 'segment_columns' is not defined

In [77]:
#Create Json data include Dataset_name, table name, columns name and columns types
result={
      'name': 'dev_test_segment_dashboard',
      'tables': [
          {'name': 'date', 'columns':date_columns},
          {'name': 'time', 'columns':time_columns}
      ]
      }


# import state to postgres 

In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(
    host="localhost",
    user="pd",
    password="iloveprimedata",
    dbname='eventify_stag',
    port=45432)

In [23]:
conn = psycopg2.connect(
    host="localhost",
    user="primedata",
    password="primedata10101!",
    database= 'eventify_stag',
    port=45432)

OperationalError: connection to server at "localhost" (::1), port 45432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 45432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?


In [8]:
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="concac123",
    port=5432)

In [4]:
cur = conn.cursor()

In [10]:
sql_query = """CREATE TABLE ManageState (
  id serial NOT NULL PRIMARY KEY,
  datasetId text NOT NULL,
  object json NOT NULL,
  timestamp timestamp default current_timestamp
)"""

In [16]:
stateObj={
    'erp':{
        'at': '2020-01-02 00:00:00'
    },
    'segment': {
        'segment_start_time': '',
        'segment_id': '',
        'user': ''
    },
    'segment_name': {
        'start_time': '',
        'end_time': '',
        'user_id': ''
    },
    'user_id': {
        'user': ''
    },
    'product':{}
}

In [11]:
cur.execute(sql_query)

In [12]:
cur.execute('select * from ManageState')

In [13]:
cur.fetchall()

[]

In [14]:
import json

In [19]:
dumped_stateObj = json.dumps(stateObj)

In [20]:
insert_query = """INSERT INTO ManageState ("datasetId,object") VALUES (%s)"""
record_insert = (obj1,)

NameError: name 'obj1' is not defined

In [ ]:
cur.execute(insert_query,record_insert)

# create dataset

In [4]:
import msal
import requests
import json

# --------------------------------------------------
# Set local variables
# --------------------------------------------------
client_id='b1e937d4-ade0-4aec-ba94-529a143c9cc1'
username = 'integration@primedata.ai'
password = 'SbSetznmvEepvEvehDnO@'
authority_url = 'https://login.microsoftonline.com/primedata.ai'
scope = ["https://analysis.windows.net/powerbi/api/.default"]
url_groups = 'https://api.powerbi.com/v1.0/myorg/groups'

# --------------------------------------------------
# Use MSAL to grab a token
# --------------------------------------------------
app = msal.PublicClientApplication(client_id, authority=authority_url)
result = app.acquire_token_by_username_password(username=username,password=password,scopes=scope)

# --------------------------------------------------
# Check if a token was obtained, grab it and call the
# Power BI REST API, otherwise throw up the error message
# --------------------------------------------------
if 'access_token' in result:
    access_token = result['access_token']
    header = {'Content-Type':'application/json','Authorization': f'Bearer {access_token}'}
    api_out = requests.get(url=url_groups, headers=header)
    print(api_out.json())
else:
    print(result.get("error"))
    print(result.get("error_description"))

{'@odata.context': 'http://wabi-south-east-asia-b-primary-redirect.analysis.windows.net/v1.0/myorg/$metadata#groups', '@odata.count': 3, 'value': [{'id': '7da7604f-20b3-49c0-aee5-6eb0c2cc010d', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'Default'}, {'id': '66871624-e0e7-4314-92ca-24e2864597b6', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'aristino'}, {'id': 'f1d27fc1-a8ca-40fa-aca9-19c1e277384f', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'dev_Test'}]}


In [5]:
s = requests.Session()

In [6]:
#s.headers['Content-Type'] = 'application/json'
s.headers['Authorization'] = f'Bearer {access_token}'

In [16]:
#Create Json data include Dataset_name, table name, columns name and columns types
result={
      'name': 'dev_test_segment_dashboard',
      'tables': [
          {'name': 'dim_date', 'columns':date_columns},
          {'name': 'dim_time', 'columns':time_columns},
          {'name': 'erp', 'columns':erp_columns},
          {'name': 'product', 'columns': product_columns},
          {'name': 'user', 'columns':[{'name': 'user_id','dataType':'string'}]},
          {'name': 'fact_segment', 'columns':fact_segment_columns},
          {'name': 'dim_segment', 'columns':dim_segment_columns}],
        "relationships": [
            {'name': 'filter_ngay_ct',
             'fromTable': 'erp',
             'fromColumn': 'ngay_ct',
             'toTable': 'dim_date',
             'toColumn': 'date',
             "crossFilteringBehavior": 'OneDirection'},
            {'name': 'filter_gio_gh',
             'fromTable': 'erp',
             'fromColumn': 'gio_gh',
             'toTable': 'dim_time',
             'toColumn': 'time',
             "crossFilteringBehavior": 'OneDirection'},
            {'name': 'purchases_product',
             'fromTable': 'erp',
             'fromColumn': 'ma_vt',
             'toTable': 'product',
             'toColumn': 'ma_vt',
             "crossFilteringBehavior": 'OneDirection'},
            {'name': 'User_buying',
             'fromTable': 'erp',
             'fromColumn': 'anonymous_id',
             'toTable': 'user',
             'toColumn': 'user_id',
             "crossFilteringBehavior": 'OneDirection'},
            {'name': 'segment_user',
             'fromTable': 'user',
             'fromColumn': 'user_id',
             'toTable': 'fact_segment',
             'toColumn': 'user_id',
             "crossFilteringBehavior": 'BothDirections'},
            {'name': 'filter_segment',
             'fromTable': 'fact_segment',
             'fromColumn': 'segment_transform',
             'toTable': 'dim_segment',
             'toColumn': 'segment_transform',
             "crossFilteringBehavior": 'OneDirection'}
              
        ]
      }


In [41]:
def create_dataset(data):
    data = json.dumps(data)
    resp = s.post('https://api.powerbi.com/v1.0/myorg/datasets',data)
    return resp

In [19]:
def create_dataset_ingroup(data,groupId):
    data = json.dumps(data)
    resp = s.post(f'https://api.powerbi.com/v1.0/myorg/groups/{groupId}/datasets',data)
    return resp

In [17]:
groupId= 'f1d27fc1-a8ca-40fa-aca9-19c1e277384f'

In [21]:
response = create_dataset_ingroup(result,groupId)
response

<Response [201]>

In [22]:
json.loads(response.text)

{'@odata.context': 'http://api.powerbi.com/v1.0/myorg/groups/f1d27fc1-a8ca-40fa-aca9-19c1e277384f/$metadata#datasets/$entity',
 'id': '4a2e081b-b0eb-4850-b481-aafe1c9a658c',
 'name': 'dev_test_segment_dashboard',
 'defaultRetentionPolicy': 'None',
 'targetStorageMode': 'Unknown',
 'upstreamDatasets': [],
 'users': []}

# save state

In [23]:
import psycopg2

In [31]:
conn = psycopg2.connect(
    host="localhost",
    user="pd",
    password="iloveprimedata",
    database = 'eventify_stag',
    port=15432)

In [32]:
new_state={
        'erp':{
            'at': '2021-01-01 00:00:00',
        },
        'dim_segment': {
            'at': '',
            'segment_id': '',
        },
        'fact_segment': {
            'at': '',
            'segment_id': '',
            'user_id': ''
        },
        'user': {
            'user_id': ''
        },
        'product':{
            'ma_vt':''
        }
        }

In [33]:
def SaveState(conn, schema, state_table,state_obj):
    obj = json.dumps(state_obj)
    try:
        with conn.cursor() as cur:
            # insert table in postgres
            sql_query = f"""INSERT INTO {schema}.{state_table} (groupId,datasetId,object) VALUES (%s,%s,%s)"""

            # insert object

            record_insert = ('f1d27fc1-a8ca-40fa-aca9-19c1e277384f','4a2e081b-b0eb-4850-b481-aafe1c9a658c',obj)

            # execute
            cur = conn.cursor()
            cur.execute(sql_query, record_insert)
            conn.commit()

            # count row insert into table

            count = cur.rowcount
    except (Exception, psycopg2.Error) as error:
        raise error

    finally:
        # closing database connection.
        conn.close()

In [34]:
SaveState(conn, 'powerbi','state_management',new_state)

In [55]:
cur = conn.cursor()

InterfaceError: connection already closed

In [98]:
cur.execute('select * from powerbi.state_management')

In [99]:
cur.fetchall()

[(2,
  '1',
  '4252e8f9-80de-4f2f-bde5-0e496459d6ca',
  {'erp': {'at': '2020-01-02 00:00:00'},
   'dim_segment': {'at_final': '', 'segment_id': ''},
   'fact_segment': {'segment_start_time': '',
    'segment_id_state': '',
    'segment_user_id_state': ''},
   'user': {'user_id_state': ''},
   'product': {}},
  datetime.datetime(2022, 1, 12, 9, 9, 58, 811675))]

In [36]:
conn.close()
cur.close()

In [86]:
datetime.now().strftime("%Y:%m:%d %H:%M:%S")

'2022:01:12 16:03:37'

In [104]:
def get_kwagrs(dictionary):
    return kwagrs


In [117]:
choose_query = {
        'erp': {
            'query': 1,
            'args': {
                'cfg': 2,
                'CH_schema': 'eventify',
                'start_time': 3,
                'end_time': 4
            }},
    'fact_segment': {
            'query': 2,
            'args': {
                'cfg': 1,
                'CH_schema': 'eventify',
                'segment_start_time': 2,
                'segment_id_state': 1,
                'segment_user_id_state': 3
            }}}

In [129]:
for table in choose_query:
    print(choose_query[table]['query'])
    print(**choose_query[table]['args'])
    print(cfg=2)

1


TypeError: 'cfg' is an invalid keyword argument for print()

In [147]:
datetime.strptime('2013-08-08 00:00:00', "%Y-%m-%d %H:%M:%S")

ValueError: time data '2013-08-08' does not match format '%Y-%m-%d %H:%M:%S'

In [148]:
stateObj={"erp": 
 {"at": "2022-01-02 00:00:00"}, 
 "dim_segment": {"at_final_state": "", 
                 "segment_id_state": ""}, 
 "fact_segment": {"segment_start_time": "", 
                  "segment_id_state": "", 
                  "segment_user_id_state": ""}, 
 "user": {"user_id_state": ""}, "product": {}}

In [150]:
for table in stateObj:
    print(table)

erp
dim_segment
fact_segment
user
product


In [48]:
erp_columns

[{'name': 'anonymous_id', 'dataType': 'string'},
 {'name': 'ngay_ct', 'dataType': 'Datetime', 'formatString': 'dd/mm/yyyy'},
 {'name': 'gio_gh', 'dataType': 'Datetime', 'formatString': 'h:nn:ss'},
 {'name': 'order_value', 'dataType': 'decimal'},
 {'name': 't_tien_sau_ck', 'dataType': 'decimal'},
 {'name': 'so_luong', 'dataType': 'decimal'},
 {'name': 'stt_rec', 'dataType': 'string'},
 {'name': 'dich_vu', 'dataType': 'string'},
 {'name': 'dvt', 'dataType': 'string'},
 {'name': 'nguon_dh', 'dataType': 'string'},
 {'name': 'status', 'dataType': 'string'},
 {'name': 'ma_vt', 'dataType': 'string'},
 {'name': 'ma_vt2', 'dataType': 'string'},
 {'name': 'ma_ct_km', 'dataType': 'string'},
 {'name': 'ten_ct_km', 'dataType': 'string'},
 {'name': 'nh_dvcs1', 'dataType': 'string'},
 {'name': 'nh_dvcs3', 'dataType': 'string'},
 {'name': 'gender', 'dataType': 'string'},
 {'name': 'Province', 'dataType': 'string'},
 {'name': 'card_tier_name', 'dataType': 'string'},
 {'name': 'interval_age', 'dataType'

In [50]:
dim_segment_columns


[{'name': 'segment_id', 'dataType': 'string'},
 {'name': 'name', 'dataType': 'string'},
 {'name': 'segment_transform', 'dataType': 'string'},
 {'name': 'at', 'dataType': 'DateTime'}]

In [96]:
df = pd.read_excel(r'C:\Users\ADMIN\Desktop\Du-lieu.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ADMIN\\Desktop\\Du-lieu.xlsx'

In [97]:
df.drop(columns='check',inplace=True)

In [101]:
df.columns = ['hd','date','employee','revenue']

In [102]:
df_grouped = df.groupby(['hd','employee']).agg({'revenue':sum}).reset_index()

In [103]:
df_grouped

,hd,employee,revenue
0,HD2029561,CH 09,85000
1,HD2029562,CH 07,60000
2,HD2029563,CH 06,50000
3,HD2029566,CH 03,139000
4,HD2029567,CH 03,97000
...,...,...,...
7469,HDO1641547026412,CH 05,100000
7470,HDO1641554618766,CH 33,66000
7471,HDO1641795736316_174150,CH 28,30000
7472,HDO1641890959869,CH 06,879000


In [105]:
df_grouped[df_grouped['revenue']>100000]

,hd,employee,revenue
3,HD2029566,CH 03,139000
7,HD2029571,CH 13,101000
8,HD2029572,CH 33,125000
16,HD2029588,CH 03,139000
33,HD2029612,CH 09,127000
...,...,...,...
7464,HDO1641546717821,CH 06,410000
7465,HDO1641546746538,CH 06,110000
7466,HDO1641546869597,CH 18,139000
7472,HDO1641890959869,CH 06,879000


In [85]:
def check(number):
    if number > 100000:
        return 'x'
    return 'none'

In [107]:
df_grouped['check'] = df_grouped['revenue'].apply(check)

In [110]:
df_new = df_grouped[df_grouped['check']=="x"].copy().reset_index(drop=True)

In [114]:
df_new.groupby('employee').agg({'hd':len}).reset_index().sort_values('employee',ascending=False)

,employee,hd
0,CH 02,29
1,CH 03,212
2,CH 04,13
3,CH 05,79
4,CH 06,138
5,CH 07,56
6,CH 09,79
7,CH 10,74
8,CH 11,22
9,CH 13,119


In [117]:
df_grouped.to_csv(r'C:\Users\ADMIN\Desktop\grouped.txt',mode = 'w', sep = '\t', header = True, index = None,encoding = 'utf-8')

In [63]:
from datetime import datetime

In [96]:
datetime.fromtimestamp(1622505600).strftime("%Y-%m-%d %H:%M:%S")

'2021-06-01 07:00:00'

In [79]:
test= datetime.strptime('2022-01-16 21:00:15',"%Y-%m-%d %H:%M:%S")

In [120]:
test2= 1622505600000000000

In [123]:
def standardized_in_state(x): # x is in [datetime,date,ctime]
    if type(x) is datetime:
        return x.strftime("%Y-%m-%d %H:%M:%S")
    if type(x) == int:
        return datetime.fromtimestamp(x/1e9).strftime("%Y-%m-%d %H:%M:%S")
    return x

In [124]:
standardized_in_state(test2)

'2021-06-01 07:00:00'

In [107]:
type(test) is datetime

True

In [115]:
isinstance(test, datetime)

True

In [118]:
int(1e9)

1000000000

In [ ]:
1622505600000000000

In [70]:
a= {
  "erp": {
    "at": "2021-01-01 00:00:00"
  },
  "dim_segment": {
    "at": "",
    "segment_id": ""
  },
  "fact_segment": {
    "at": "",
    "segment_id": "",
    "user_id": ""
  },
  "user": {
    "user_id": ""
  },
  "product": {
    "ma_vt": ""
  }
}

In [72]:
import copy
b = copy.deepcopy(a)

In [73]:
b

{'erp': {'at': '2021-01-01 00:00:00'},
 'dim_segment': {'at': '', 'segment_id': ''},
 'fact_segment': {'at': '', 'segment_id': '', 'user_id': ''},
 'user': {'user_id': ''},
 'product': {'ma_vt': ''}}

In [74]:
b['dim_segment']['segment_id']=1

In [75]:
b

{'erp': {'at': '2021-01-01 00:00:00'},
 'dim_segment': {'at': '', 'segment_id': 1},
 'fact_segment': {'at': '', 'segment_id': '', 'user_id': ''},
 'user': {'user_id': ''},
 'product': {'ma_vt': ''}}

In [76]:
a

{'erp': {'at': '2021-01-01 00:00:00'},
 'dim_segment': {'at': '', 'segment_id': ''},
 'fact_segment': {'at': '', 'segment_id': '', 'user_id': ''},
 'user': {'user_id': ''},
 'product': {'ma_vt': ''}}